# Scraping de URLs de LinkedIn de las empresas de la Ley

Una vez que armamos la base con las empresas anotadas en las distintas versiones de la Ley de Software, buscamos sus URLs de LinkedIn mediante búsquedas de Google. La búsqueda se hace a través de la razón social que estas compañías presentan en el [Régimen de Promoción de la Economía del Conocimiento](https://www.argentina.gob.ar/servicio/acceder-los-beneficios-del-regimen-de-promocion-de-la-economia-del-conocimiento). Dado que se busca a través de la razón social y este puede ser muy distinto al nombre de fantasía, puede que muchas no se encuentren de esta manera. Aquellas que no fueron encontradas son posteriormente buscadas manualmente. Además, se revisaron los links encontrados y muchos fueron corregidos por referirse a una persona o a otras compañías. 

In [1]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.keys import Keys  # For sending key presses
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By 
import time

In [2]:
DATASETS_DIR = '../datasets/'

In [3]:
empresas_regimen = pd.read_csv(DATASETS_DIR + 'Empresas_LPS.csv')
empresas_regimen.head()

,Unnamed: 0,Cuit,Empresa / Razón Social,Región,Provincia,Localidad,Número De Acto,Fecha Del Acto Administrativo,Fecha En Boletín Oficial,tamaño
0,0,30715253727,Brave New Technology SRL,Pampeana,Ciudad Autonoma Buenos Aires,Ciudad Autonoma De Buenos Aires,533E,15/8/2018,16/8/2018,NaN
1,1,30712485546,Bright Sector SRL,Pampeana,Ciudad Autonoma Buenos Aires,Ciudad Autonoma De Buenos Aires,99,13/8/2015,20/8/2015,NaN
2,2,30708928387,Bsf SA,Cuyo,Mendoza,Chacras De Coria,155,17/11/2015,23/11/2015,NaN
3,3,30651515897,Btb Soft SA,Pampeana,Ciudad Autonoma Buenos Aires,Ciudad Autonoma De Buenos Aires,7,29/2/2016,3/4/2016 00:00,NaN
4,4,30607986289,Buenosaires Software SA,Pampeana,Ciudad Autonoma Buenos Aires,Ciudad Autonoma De Buenos Aires,41,15/6/2016,22/6/2016,NaN


In [23]:
# Utilizo un chromedriver para acceder a google
chrome_service = Service("chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=chrome_service)

# Busco a través de cada uno de los nombres de las compañías y las busco en google con esa razón social. Luego guardo en el dataframe el link.

for index, row in empresas_regimen.iterrows():
    company_name = row['Empresa / Razón Social']    
    search_query = f"{company_name} linkedin"
    google_url = f"https://www.google.com/search?q={search_query}"
    
    driver.get(google_url)
    
    try:
        search_results = driver.find_elements(By.CSS_SELECTOR, "div.g")
        for result in search_results:
            
            link = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            if "linkedin.com" in link:
                empresas_regimen.at[index, 'LinkedIn'] = link
                break  # Salto a lo próximo si no encuentra un link de LinkedIn
        else:
            empresas_regimen.at[index, 'LinkedIn'] = "Not found"
        
        time.sleep(5)
    except Exception as e:
        print(f"Exception occurred: {e}")
        empresas_regimen.at[index, 'LinkedIn'] = "Not found"
        time.sleep(5)

driver.quit()

In [24]:
empresas_regimen.head()

,Cuit,Empresa / Razón Social,Región,Provincia,Localidad,Número De Acto,Fecha Del Acto Administrativo,Fecha En Boletín Oficial,LinkedIn
0,30715253727,Brave New Technology SRL,Pampeana,Ciudad Autonoma Buenos Aires,Ciudad Autonoma De Buenos Aires,533E,15/8/2018,16/8/2018,https://ar.linkedin.com/in/tiendas-new-technol...
1,30712485546,Bright Sector SRL,Pampeana,Ciudad Autonoma Buenos Aires,Ciudad Autonoma De Buenos Aires,99,13/8/2015,20/8/2015,https://ar.linkedin.com/in/luc%C3%ADaalegria
2,30708928387,Bsf SA,Cuyo,Mendoza,Chacras De Coria,155,17/11/2015,23/11/2015,https://ar.linkedin.com/company/bsf
3,30651515897,Btb Soft SA,Pampeana,Ciudad Autonoma Buenos Aires,Ciudad Autonoma De Buenos Aires,7,29/2/2016,2016-04-03 00:00:00,https://ar.linkedin.com/company/btb-software?t...
4,30607986289,Buenosaires Software SA,Pampeana,Ciudad Autonoma Buenos Aires,Ciudad Autonoma De Buenos Aires,41,15/6/2016,22/6/2016,https://ar.linkedin.com/company/buenosairessof...


In [25]:
empresas_regimen.to_csv(DATASETS_DIR + 'Empresas_LPS_URLs.csv')